# Поиск вырубок: разведка

Цель всех действий, которые производятся ниже -- понять, как выглядят вырубки на снимках, и какими призаками можно пользоваться, чтобы эти вырубки "поймать".

Пока я вижу следующие потенциально рабочие подходы:
 
1. Анализ временной динамики -- вырубка это резкое изменение отражательной способности. Причем изменение не любое, а определенного вида. Такой подход подразумевает, что мы можем обрабатывать каждый пиксель по одтельности, строить соответсвующие пикселю временные ряды изменений и искать на них "типичные" для вырубок изменения. Подход, по видимости, требует качественной фильтрации данных от мусора (облаков, теней) и уравнивания (стандартизации) снимков, сделанных в разное время (разное время года, разная освещенность, разные атмосферные условия). Представляется, что подобная предобработка данных будет довольно трудоемкой. *Текущий подход, который использует метрики, по сути является вариацией этого метода: метрики как раз и дают относительно устойчивые характеристики территории, которые сглаживают различия разновременных снимков.*
2. Анализ данных в пространственной области. Тут имеется в виду, что на снимках высокого разрешнения вырубки могут иметь характерный вид: центральная часть и идущие к ней волоки. Если это действительно так, то можно попробовать определять вырубки по подобным "отпечаткам". Это позволит избежать сложностей со стандартизацией разновременных снимков. С другой стороны, такой подход, вероятно, даст большое число ложных срабатываний на участках, которые похожи на вырубки (пролысины?, перекрестки дорог?). Тем не менее, мне представляется, что этот подход проще первого в реализации и, хотя первый путь потенциально более перспективен, я думаю, что нужно начать с поиска в пространственной области (легче реализовать => быстрее получим результаты => в случае удачи сэкономим время и усилия, а в случае неудачи потратимся не очень значительно).

Также возможны гибридные варианты, когда вырубка ищется и в пространственной, и во временной области.

Поэтому первая часть блокнота содержит раздел по визуализации имеющихся вырубок и визуального выделения признаков вырубок.

# Комментарий ИГ
1. Пока задача стоит разобратся с одним временем года (декабрь-март) и это с одной стороны проще - поскольку, нет фенологических изменений (за редким исключением), с другой стороны это сужает (возможно, на самом деле может быть и нет) возможности использования всяких индексов (NDVI и пр.). По первому подходу я вижу следующую последовательность действий (если это можно реализовать на данной платформе):

А) Создание базового безоблочного композита для зимы 2015 г. (если не получится набрать достаточное кол-во данных, добавить 2014). Здесь нам важно получить максимальное количество безоблачных пикселей и полное покрытие территории. Время отдельной сцены не важно.
Получение снимков и перевод в TOA - реализовано

QA анализ и нормализация - удаление облаков и теней на первом этапе можно использовать BQA слой, но думаю, что качество не будет нас устраивать, можно попробовать добавить буфера (элементы best pixels selection -http://www.tandfonline.com/doi/full/10.1080/07038992.2014.945827). 

Другие варианты - добавить FMASK (если это можно сделать на этой платформе) или сделать отдельный модуль для классификации на облака/воду/тени и т.д. Я могу сделать нужный слой треннингов для терриитории.

Про нормализацию: сначала, можно попробовать сделать вообще без нее (главное без облаков) и посмотреть что получится, но если возникнет необходимость добавлять данные (L7,S2), то понадобится. 

Если следовать "правильному" пути, то нужно сделать два модуля:

1) пересчитывает значения в каналах в зависимости от положения от центра снимка (можно просто взять коэф. регрессии из Петиной статьи, например, поскольку этот параметр вряд ли должен менятся в зав. от сезона) 
2) Нормализация по МОДИС (MOD43 или MOD09) -отбор "лучших" пикселей и пересчет (в среднее или медианное) значение за этот же период (2015) - построение ур. регрессии (MOD ~ LANDSAT) -коэф. использовать при обработке.

Создание композита.

Б) Обработка актуальных данных
Получение снимков и перевод в TOA - реализовано

QA анализ и нормализация - здесь пока можно ограничится только удалением облаков, поскольку изменения (рубки) в большинстве случаев сильно перекрывают всю прочую вариацию

Классификация

2. По второму подходу комментариев у меня пока нет

Планируется использовать -какие то текстурные характеристики брать с VHR и применять к Landsat -у?


## Подключение необходимых функций

Импортируем необходимые библиотеки и функции:

In [1]:
import utilites
reload(utilites)

from utilites import (
    get_grassdata_path,
    get_location_name,
    get_location_path,
    get_wms_link,
)

In [2]:
from grasslib import GRASS

grs = GRASS(gisbase='/usr/lib/grass70', 
            dbase=get_grassdata_path(), 
            location=get_location_name()
)

Для того, чтобы не засорять PERMANENT вспомогательными и/или временными данными, переключимся в отдельный MAPSET (под названием exploration). Если такого набора нет, то он будет создан (флаг -c):

In [3]:
grs.grass.run_command('g.mapset', mapset='exploration', flags='c')
print grs.grass.read_command('g.mapset', flags='p')

exploration



## Как выглядят вырубки?

Для удобства отображения снимков воспользуемся WMS-сервисом. На снимки можно ссылаться следующим образом:

```
<img src="http://176.9.38.120/cruncher_wms/wms/landsat/?SERVICE=WMS&REQUEST=GetMap&BBOX=MinX,MinY,MaxX,MaxY&WIDTH=...&HEIGHT=...&LAYERS=...">
```

Например:
```
<img src="http://176.9.38.120/cruncher_wms/wms/landsat/?SERVICE=WMS&REQUEST=GetMap&BBOX=516684,5098286,543339.0,5113090.0&WIDTH=200&HEIGHT=200&LAYERS=toar_LC81120282015237LGN00_B5@landsat">
```


Снимок1 | Снимок2
--------|---------
<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=516684,5098286,543339.0,5113090.0&WIDTH=200&HEIGHT=200&LAYERS=toar_LC81120282015237LGN00_B4@landsat">|<img src="http://176.9.38.120/cruncher_wms/wms/landsat/?SERVICE=WMS&REQUEST=GetMap&BBOX=516684,5098286,543339.0,5113090.0&WIDTH=200&HEIGHT=200&LAYERS=toar_LC81120282015237LGN00_B5@landsat">

Генерировать сылки можно при помощи вспомогательной функции:

In [18]:
print get_wms_link('toar_LC81120282015237LGN00_B5@landsat', 516684,5098286,543339.0,5113090.0)

http://176.9.38.120/cruncher_wms/wms/landsat/?SERVICE=WMS&REQUEST=GetMap&BBOX=516684,5098286,543339.0,5113090.0&WIDTH=300&HEIGHT=300&LAYERS=toar_LC81120282015237LGN00_B5@landsat


### Рубки в PlanetLabs

Вырубка, появившаяся зимой в промежутке между 8-го января и 3-го февраля 2016-го года. Вырубка едва заметна. Найти ее можно по более светлому пятну по сравнению с окружающим его участком. Сам участок помимо того, что поменял цвет, стал более "шершавый" на вид.

[До вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15057614.153513663&center=6043173.859893657&resolution=1.194328566955879&styles=16,29) | [После вырубки](http://kolesov.nextgis.com/resource/1/display?base=osm-mapnik&center=15057614.153513663&center=6043173.859893657&resolution=1.194328566955879&styles=16,23)
--------|---------
<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=519572,5273372,520259,5273908&WIDTH=600&HEIGHT=600&LAYERS=20160108_011229_0c53_visual.composite@planet">|<img src="http://176.9.38.120/cruncher_wms/wms/planet/?SERVICE=WMS&REQUEST=GetMap&BBOX=519572,5273372,520259,5273908&WIDTH=600&HEIGHT=600&LAYERS=20160302_225400_0c68_visual.composite@planet">

